In [2]:
import SimpleITK as sitk
import pims
# from nd2reader import ND2Reader
import numpy as np
import matplotlib.pyplot as plt
import sys

sys.path.insert(0,'/home/jmamede/scripts')
# from support_pla import othercolor, cell_mask, multiply, rebin
from support_pla import convert16to8bits
import tifffile
import cv2  # OpenCV for fast resizing



from glob import glob

from pystackreg import StackReg
# from skimage import io

def rebin(arr, new_shape):
    from PIL import Image
    return np.array(Image.fromarray(arr).resize(new_shape,resample=Image.NEAREST))

def calculate_reg(ref,mov,name):
    from pystackreg import StackReg
    transformations = {
        'TRANSLATION': StackReg.TRANSLATION,
        'RIGID_BODY': StackReg.RIGID_BODY,
        'SCALED_ROTATION': StackReg.SCALED_ROTATION,
        'AFFINE': StackReg.AFFINE,
        'BILINEAR': StackReg.BILINEAR
    }
    sr = StackReg(transformations[name])
    return sr,sr.register(ref, mov,)

def registration(image,tmat,name='AFFINE'):
    from pystackreg import StackReg
    from skimage import transform, io, exposure
    transformations = {
        'TRANSLATION': StackReg.TRANSLATION,
        'RIGID_BODY': StackReg.RIGID_BODY,
        'SCALED_ROTATION': StackReg.SCALED_ROTATION,
        'AFFINE': StackReg.AFFINE,
        'BILINEAR': StackReg.BILINEAR
    }
    sr = StackReg(transformations[name])
    return sr.transform(image,tmat)
    

In [3]:
ficheiros = glob('/home/jmamede/Data/Janet/mIF/20211122mIFOrganoids/*tif')

In [4]:
ficheiros.sort()
ficheiros

['/home/jmamede/Data/Janet/mIF/20211122mIFOrganoids/C0-DAPI-InfectedSNs.ome.tif',
 '/home/jmamede/Data/Janet/mIF/20211122mIFOrganoids/C0-GFAP-InfectedSNs.ome.tif',
 '/home/jmamede/Data/Janet/mIF/20211122mIFOrganoids/C0-S100b-InfectedSNs.ome.tif',
 '/home/jmamede/Data/Janet/mIF/20211122mIFOrganoids/C0-Tuj1-InfectedSNs.ome.tif',
 '/home/jmamede/Data/Janet/mIF/20211122mIFOrganoids/C1-DAPI-InfectedSNs.ome.tif',
 '/home/jmamede/Data/Janet/mIF/20211122mIFOrganoids/C1-Ecad-InfectedSNs.ome.tif',
 '/home/jmamede/Data/Janet/mIF/20211122mIFOrganoids/C1-Iba1-InfectedSNs.ome.tif',
 '/home/jmamede/Data/Janet/mIF/20211122mIFOrganoids/C1-cGAS-InfectedSNs.ome.tif',
 '/home/jmamede/Data/Janet/mIF/20211122mIFOrganoids/C2-CD56-InfectedSNs.ome.tif',
 '/home/jmamede/Data/Janet/mIF/20211122mIFOrganoids/C2-CD69-InfectedSNs.ome.tif',
 '/home/jmamede/Data/Janet/mIF/20211122mIFOrganoids/C2-DAPI-InfectedSNs.ome.tif',
 '/home/jmamede/Data/Janet/mIF/20211122mIFOrganoids/C2-HLAdr-InfectedSNs.ome.tif']

In [5]:
ficheiros = ficheiros[:5]
ficheiros

['/home/jmamede/Data/Janet/mIF/20211122mIFOrganoids/C0-DAPI-InfectedSNs.ome.tif',
 '/home/jmamede/Data/Janet/mIF/20211122mIFOrganoids/C0-GFAP-InfectedSNs.ome.tif',
 '/home/jmamede/Data/Janet/mIF/20211122mIFOrganoids/C0-S100b-InfectedSNs.ome.tif',
 '/home/jmamede/Data/Janet/mIF/20211122mIFOrganoids/C0-Tuj1-InfectedSNs.ome.tif',
 '/home/jmamede/Data/Janet/mIF/20211122mIFOrganoids/C1-DAPI-InfectedSNs.ome.tif']

In [4]:

image = tifffile.imread(ficheiros[0], key=0)
h, w = image.shape
image.shape

(19813, 35570)

In [5]:
len(ficheiros)

13

In [6]:
new = np.zeros((1,len(ficheiros)+1,h,w,))
new = new.astype(np.uint8)

In [7]:
for i,_file in enumerate(ficheiros):
    new[0,i,:,:] = tifffile.imread(_file, key=0).astype(np.uint8)

In [8]:
print((new[0,4]).mean(),new[0,4].mean())

7.707354582041554 7.707354582041554


In [9]:
new[0,4,...].shape

(19813, 35570)

In [10]:
new[0,13,...] = (new[0,4,...]+10) - new[0,2,...]



In [11]:

new = new.astype(np.uint8)
new.dtype

dtype('uint8')

In [12]:
new.shape

(1, 14, 19813, 35570)

In [27]:
import napari
v = napari.Viewer(ndisplay=2,)
# mov = np.roll(mov,-2305,axis=1)
# mov = np.roll(mov,100,axis=0)
clrs = ['red','green','blue','cyan','magenta','bop purple','yellow','bop orange']

for i in range(new.shape[0]):
    v.add_image(new[i],blending='additive',
                scale=[0.108,0.108],
                colormap=clrs[
                       np.random.randint(len(clrs))])
# v.add_image(ref,blending='additive',colormap='green')
# v.add_image(mov,blending='additive',colormap='cyan')

/home/jmamede/anaconda3/lib/python3.7/site-packages/napari/_vispy/vispy_image_layer.py:186: UserWarning: data shape (19813, 35570) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  ndisplay=self.layer._ndisplay,
/home/jmamede/anaconda3/lib/python3.7/site-packages/napari/_vispy/vispy_image_layer.py:186: UserWarning: data shape (19813, 35570) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  ndisplay=self.layer._ndisplay,
/home/jmamede/anaconda3/lib/python3.7/site-packages/napari/_vispy/vispy_image_layer.py:186: UserWarning: data shape (19813, 35570) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  ndisplay=self.layer._ndisplay,
/home/jmamede/anaconda3/lib/python3.7/site-packages/napari/_vispy/vispy_image_layer.py:186: UserWarning: data shape (19813, 35570) exceeds GL_MAX_TEXTURE_SIZE 1638

/home/jmamede/anaconda3/lib/python3.7/site-packages/napari/_vispy/vispy_image_layer.py:186: UserWarning: data shape (35570, 19813) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  ndisplay=self.layer._ndisplay,
/home/jmamede/anaconda3/lib/python3.7/site-packages/napari/_vispy/vispy_image_layer.py:186: UserWarning: data shape (35570, 19813) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  ndisplay=self.layer._ndisplay,
/home/jmamede/anaconda3/lib/python3.7/site-packages/napari/_vispy/vispy_image_layer.py:186: UserWarning: data shape (19813, 35570) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  ndisplay=self.layer._ndisplay,
/home/jmamede/anaconda3/lib/python3.7/site-packages/napari/_vispy/vispy_image_layer.py:186: UserWarning: data shape (19813, 35570) exceeds GL_MAX_TEXTURE_SIZE 1638

In [13]:
names = ['cGAMP','YBX1','preGFP','DAPI','postGFP','CD4','CD69','CPSF6','DAPI','241D','AG3','SIM4','DAPI','GFP-Vpr']

In [24]:
with tifffile.TiffWriter('/run/media/jmamede/Joao/multiplex/ENDO/out/all.ome.tif', bigtiff=True) as tif:
    for i in range(new.shape[1]):
        options = dict(
             tile=(4096, 4096),
    #         imagej=False,
            photometric='minisblack',
            compress=9,
            resolution=(0.108, 0.108),
            metadata={'axes': 'YX',
                'Channel': {'Name': names[i]}}
        )

#     for ch in new.shape[0]:
        tif.write(new[0,i,...].astype(np.uint8),
    #                         compression='ZLIB',
                          **options
                            )
        


/home/jmamede/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: TiffWriter: the 'compress' parameter is deprecated since 2020.9.30. Use the 'compression' parameter
  app.launch_new_instance()


In [19]:
with tifffile.TiffWriter('/run/media/jmamede/Joao/multiplex/ENDO/out/all_chpiramid.ome.tif', bigtiff=True) as tif:
    options = dict(
             tile=(2048, 2048),
    #         imagej=False,
            photometric='minisblack',
#             compress=9,
            resolution=(0.108, 0.108),
            metadata={'axes': 'ZCYX',
                'Channel': {'Name': names}}
        )
    tif.write(new,
               subifds=3, 
               **options)     
#     for i,_file in enumerate(ficheiros):
    tif.write(new[:,:,::4,::4], subfiletype=1, **options)
    tif.write(new[:,:,::8,::8], subfiletype=1, **options)
    tif.write(new[:,:,::16,::16], subfiletype=1, **options)
                        
                        
        
tif.close()

In [25]:
del new

In [10]:

     tif.write(new,
               subifds=2, 
               **options)
     # save pyramid levels to the two subifds
     # in production use resampling to generate sub-resolutirons
#      tif.write(new[::2, ::2], subfiletype=1, **options)
#      tif.write(new[::4, ::4], subfiletype=1, **options)
     tif.write(new[:,:,::8, ::8], subfiletype=1, **options)
#      tif.write(new[::16, ::16], subfiletype=1, **options)

ValueError: data too large for standard TIFF file

In [14]:
tif.close()
del new

In [18]:
new.shape[0]

6

In [24]:
mov_roll = np.roll(mov,-35,axis=1)
mov_roll = np.roll(mov_roll,0,axis=0)


In [25]:

v.add_image(mov_roll,blending='additive',colormap='red')

/home/jmamede/anaconda3/lib/python3.7/site-packages/napari/_vispy/vispy_image_layer.py:186: UserWarning: data shape (19813, 35570) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  ndisplay=self.layer._ndisplay,


<Image layer 'mov_roll' at 0x7f2213920750>

In [ ]:
# test1 = sr.transform(mov,tmat=mta1)
#Translational transformation
sr = StackReg(StackReg.RIGID_BODY)
mta1 = sr.register(ref,mov
                  )

In [ ]:

#Translational transformation
sr = StackReg(StackReg.AFFINE)
mta1 = sr.register(ref[0],mov[0])

In [ ]:
mta1

In [ ]:
mta1 = np.array([
       [ 9.99999951e-01, -3.12420350e-04,  3.28717291e+02],
       [ 3.12420350e-04,  9.99999951e-01, -3.98938467e+01],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]]
)

In [8]:
mta1 = np.array([[ 1.00108756e+00,  1.24853472e-04, -1.21100576e+01],
       [ 3.28726193e-03,  1.00099829e+00, -4.27326598e+01],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

In [18]:
import napari
v = napari.Viewer(ndisplay=2)

Reader will load image in-memory: False


/home/jmamede/anaconda3/lib/python3.7/site-packages/napari/_vispy/vispy_image_layer.py:186: UserWarning: data shape (19813, 35570) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  ndisplay=self.layer._ndisplay,


Reader will load image in-memory: False


/home/jmamede/anaconda3/lib/python3.7/site-packages/napari/_vispy/vispy_image_layer.py:186: UserWarning: data shape (19813, 33534) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  ndisplay=self.layer._ndisplay,


Reader will load image in-memory: False


/home/jmamede/anaconda3/lib/python3.7/site-packages/napari/_vispy/vispy_image_layer.py:186: UserWarning: data shape (19813, 33534) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  ndisplay=self.layer._ndisplay,
/home/jmamede/anaconda3/lib/python3.7/site-packages/napari/_vispy/vispy_image_layer.py:186: UserWarning: data shape (19813, 35570) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  ndisplay=self.layer._ndisplay,
/home/jmamede/anaconda3/lib/python3.7/site-packages/napari/_vispy/vispy_image_layer.py:186: UserWarning: data shape (19813, 33534) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  ndisplay=self.layer._ndisplay,
/home/jmamede/anaconda3/lib/python3.7/site-packages/napari/_vispy/vispy_image_layer.py:186: UserWarning: data shape (19813, 33534) exceeds GL_MAX_TEXTURE_SIZE 1638

/home/jmamede/anaconda3/lib/python3.7/site-packages/napari/_vispy/vispy_image_layer.py:186: UserWarning: data shape (19813, 33534) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  ndisplay=self.layer._ndisplay,
/home/jmamede/anaconda3/lib/python3.7/site-packages/napari/_vispy/vispy_image_layer.py:186: UserWarning: data shape (19813, 33534) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  ndisplay=self.layer._ndisplay,


In [14]:
v.add_image(ref2,blending='additive',colormap='green')
v.add_image(mov2,blending='additive',colormap='red')
v.add_image(test1,blending='additive',colormap='cyan')

/home/jmamede/anaconda3/lib/python3.7/site-packages/napari/_vispy/vispy_image_layer.py:186: UserWarning: data shape (19813, 35570) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  ndisplay=self.layer._ndisplay,
/home/jmamede/anaconda3/lib/python3.7/site-packages/napari/_vispy/vispy_image_layer.py:186: UserWarning: data shape (19813, 35570) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  ndisplay=self.layer._ndisplay,
/home/jmamede/anaconda3/lib/python3.7/site-packages/napari/_vispy/vispy_image_layer.py:186: UserWarning: data shape (19813, 35570) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  ndisplay=self.layer._ndisplay,


<Image layer 'test1' at 0x7f457da9a350>

In [27]:
mov
mov2[:mov.frame_shape[0],:mov.frame_shape[1]] = mov[dapich]

NameError: name 'test1' is not defined

In [7]:
ficheiros = glob('/run/media/jmamede/Joao/multiplex/ENDO/out/*8bit*tiff')
ficheiros.sort()

#load reference and "moved" image
ref = pims.open(ficheiros[0])
mov = pims.open(ficheiros[1])


sizes = np.array([mov.frame_shape,ref.frame_shape]).T
empty = sizes[0].max(),sizes[1].max()

ref2 = np.zeros(empty )
mov2 = np.zeros(empty )


sr = StackReg(StackReg.AFFINE)

In [8]:
# quarter = tuple(np.round(np.array(ref2.shape)/5).astype('int'))


# mta3 = sr.register(
#     ref2[quarter[0]:-quarter[0],quarter[1]:-quarter[1]],
#     mov2[quarter[0]:-quarter[0],quarter[1]:-quarter[1]]
#                              )

In [9]:
# mta3 = np.array([[ 1.00108756e+00,  1.24853472e-04, -1.21100576e+01],
#        [ 3.28726193e-03,  1.00099829e+00, -4.27326598e+01],
#        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

mta3 = np.array([[ 9.98186010e-01, -9.85277006e-04,  1.08630602e+01],
       [ 9.66418193e-04,  9.97938822e-01,  8.23410514e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

In [14]:
mov2.reshape((1,1,)+(mov2.shape)).shape

(1, 19813, 35570)

In [17]:
tif.close()

In [8]:
ref2 = rebin(ref2,(ref2.shape[0]//2,ref2.shape[1]//2))
mov2 = rebin(mov2,(mov2.shape[0]//2,mov2.shape[1]//2) )

In [9]:
ref2.shape

(17785, 9907)

In [10]:
%timeit -n1 -r 2 print('asdf')

asdf
asdf
63.4 µs ± 25.2 µs per loop (mean ± std. dev. of 2 runs, 1 loop each)


In [11]:

#Translational transformation
sr = StackReg(StackReg.TRANSLATION)
out_tra = sr.register(ref2, mov2)

In [13]:
sr2 = StackReg(StackReg.TRANSLATION)
out_tra2 = sr2.register(ref[0][4940:14818,8893:26678], mov[0][4940:14818,8893:26678])

In [5]:
out_tra = np.array([[  1.        ,   0.        , 641.24722516],
       [  0.        ,   1.        , -83.64280372],
       [  0.        ,   0.        ,   1.        ]])


In [6]:
out_tra2 = np.array([[  1.        ,   0.        , 641.24722516*2],
       [  0.        ,   1.        , -83.64280372*2],
       [  0.        ,   0.        ,   1.        ]])

In [8]:
ref2 = np.zeros((19814,35570))
mov2 = np.zeros((19814,35570))
ref2[:ref.frame_shape[0],:ref.frame_shape[1]] = ref[0]
mov2[:mov.frame_shape[0],:mov.frame_shape[1]] = mov[0]

sr = StackReg(StackReg.TRANSLATION)
test1 = sr.transform(mov2,tmat=out_tra)

In [11]:
s2 = StackReg(StackReg.TRANSLATION)
test2 = s2.transform(mov[0],tmat=out_tra2)

In [20]:
#Rigid Body transformation
sr = StackReg(StackReg.RIGID_BODY)
out_rot = sr.register(ref2, mov2)

In [51]:
out_rot = out_rot*2

In [ ]:
out_rot[2][2] = 1.0
out_rot
test2 = sr.transform(mov2,tmat=out_rot)

In [59]:
out_rot[2][2] = 1
out_rot

array([[ 1.99999983e+00, -8.24006515e-04,  1.28852116e+03],
       [ 8.24006515e-04,  1.99999983e+00, -1.70715351e+02],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

In [40]:
#Scaled Rotation transformation
sr = StackReg(StackReg.SCALED_ROTATION)
out_sca = sr.register(ref2, mov2)

In [ ]:
out_sca.shape

In [29]:
#Affine transformation
sr = StackReg(StackReg.AFFINE)
out_aff = sr.register(ref2, mov2)


In [36]:
out_aff[1]

array([ 3.27281430e-01,  1.13811964e+00, -2.77346128e+03])

In [ ]:
#Bilinear transformation
sr = StackReg(StackReg.BILINEAR)
out_bil = sr.register(ref2, mov2)

In [ ]:
out_bil.shape

In [43]:
ref2.shape

(19814, 35570)

In [19]:
# reg_im = np.zeros((19814, 35570,4))
# reg_im[...,0] = ref2
# reg_im[...,1] = mov2
# reg_im[...,2] = test1
# reg_im[...,3] = test2

# reg_im[...,3] = out_rot 
# reg_im[...,4] = out_sca
# reg_im[...,5] = out_aff
# reg_im[...,6] = out_tra
# reg_im[...,7] = out_bil

ValueError: could not broadcast input array from shape (19759,35570) into shape (19814,35570)

In [13]:


cmap = ['green','red','cyan','red','red','red','red','red','red','red']

for i,img in enumerate([ref[0],mov[0],test1,test2]):
    v.add_image(img,name='image'+str(i),blending='additive',colormap=cmap[i])

/home/jmamede/anaconda3/lib/python3.7/site-packages/napari/_vispy/vispy_image_layer.py:186: UserWarning: data shape (19813, 33534) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  ndisplay=self.layer._ndisplay,
/home/jmamede/anaconda3/lib/python3.7/site-packages/napari/_vispy/vispy_image_layer.py:186: UserWarning: data shape (19759, 35570) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  ndisplay=self.layer._ndisplay,
/home/jmamede/anaconda3/lib/python3.7/site-packages/napari/_vispy/vispy_image_layer.py:186: UserWarning: data shape (19814, 35570) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  ndisplay=self.layer._ndisplay,
/home/jmamede/anaconda3/lib/python3.7/site-packages/napari/_vispy/vispy_image_layer.py:186: UserWarning: data shape (19759, 35570) exceeds GL_MAX_TEXTURE_SIZE 1638

In [48]:
v.add_image(reg_im[...,2],name='image'+str(i),blending='additive',colormap=cmap[i])

/home/jmamede/anaconda3/lib/python3.7/site-packages/napari/_vispy/vispy_image_layer.py:186: UserWarning: data shape (19814, 35570) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  ndisplay=self.layer._ndisplay,


<Image layer 'image1 [1]' at 0x7f8d843c15d0>

/home/jmamede/anaconda3/lib/python3.7/site-packages/napari/_vispy/vispy_image_layer.py:186: UserWarning: data shape (19814, 35570) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  ndisplay=self.layer._ndisplay,
/home/jmamede/anaconda3/lib/python3.7/site-packages/napari/_vispy/vispy_image_layer.py:186: UserWarning: data shape (19814, 35570) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  ndisplay=self.layer._ndisplay,
/home/jmamede/anaconda3/lib/python3.7/site-packages/napari/_vispy/vispy_image_layer.py:186: UserWarning: data shape (19814, 35570) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  ndisplay=self.layer._ndisplay,
/home/jmamede/anaconda3/lib/python3.7/site-packages/napari/_vispy/vispy_image_layer.py:186: UserWarning: data shape (19814, 35570) exceeds GL_MAX_TEXTURE_SIZE 1638

In [39]:
import pims
# from nd2reader import ND2Reader
import numpy as np
import matplotlib.pyplot as plt
import sys

sys.path.insert(0,'/home/jmamede/scripts')
# from support_pla import othercolor, cell_mask, multiply, rebin
from support_pla import convert16to8bits
from pystackreg import StackReg

# from skimage import io

#load reference and "moved" image

reg_channel = 3

image0 = pims.open('/home/jmamede/Data/multiplex/KOTHP1/DAPI-iGFP-INmruby3-bactin647_v01_PRJ.Custom.ome.tiff')
image1 = pims.open('/home/jmamede/Data/multiplex/KOTHP1/DAPI-1a11_488-241_647_v01_PRJ.Custom.ome.tiff')

ref = image0[reg_channel]
mov = image1[reg_channel]

/home/jmamede/anaconda3/lib/python3.7/site-packages/pims/api.py:207: UserWarning: <class 'pims.tiff_stack.TiffStack_tifffile'> errored: 'NoneType' object has no attribute 'TiffFile'
  warn(message)


In [40]:
def calculate_reg(ref,mov,name):
    from pystackreg import StackReg
    transformations = {
        'TRANSLATION': StackReg.TRANSLATION,
        'RIGID_BODY': StackReg.RIGID_BODY,
        'SCALED_ROTATION': StackReg.SCALED_ROTATION,
        'AFFINE': StackReg.AFFINE,
        'BILINEAR': StackReg.BILINEAR
    }
    sr = StackReg(transformations[name])
    return sr.register(ref, mov,)

def registration(image,tmat,name='AFFINE'):
    from pystackreg import StackReg
    from skimage import transform, io, exposure
    transformations = {
        'TRANSLATION': StackReg.TRANSLATION,
        'RIGID_BODY': StackReg.RIGID_BODY,
        'SCALED_ROTATION': StackReg.SCALED_ROTATION,
        'AFFINE': StackReg.AFFINE,
        'BILINEAR': StackReg.BILINEAR
    }

    sr = StackReg(transformations[name])
    return sr.transform(image,tmat)
    
# def registration_gpu(image,tmat):
#     #it's not working it's inverted....
#     import cupy as cp
#     from cupyx.scipy.ndimage import affine_transform
    
#     return cp.asnumpy(
#         affine_transform(
#             cp.asarray(image),
#             cp.asarray(tmat)
#         )
#     )

matrix = calculate_reg(ref,mov,'AFFINE')


In [59]:
image1.sizes
result = np.zeros((image1.sizes['t'],image1.sizes['x'],image1.sizes['y']))

for i in range(image1.sizes['t']):
    result[i,:,:] = registration(image1[i],matrix,'AFFINE')
# result_gpu = registration_gpu(mov,matrix)

In [77]:
import napari
from vispy.color import Colormap
v = napari.Viewer(ndisplay=2)
# cmap = Colormap([[1, 0, 0], [0, 0, 0], [0, 0, 1]])
# cmap = ['green','red','red','bluered','red','red','red','red','red','red']
cmap = ['magenta','red','green','blue']

names = ['B-actin','IN-mruby3','iGFP','DAPI','CA-241','Blank','PQBP1','DAPI2']
j = 0
for img in [image0,result]:
    for i in range(4):
        v.add_image(img[i],name=names[j],blending='additive',
                    colormap=cmap[i]                    
#                     colormap=('diverging', cmap)
                   )
        j +=1